# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 29 2022 10:37AM,240721,19,8208642,VITABIOTICS USA INC.,Released
1,Jun 29 2022 10:35AM,240719,10,MSP211725,"Methapharm, Inc.",Released
2,Jun 29 2022 10:35AM,240719,10,MSP211728,"Methapharm, Inc.",Released
3,Jun 29 2022 10:35AM,240719,10,MSP211733,"Methapharm, Inc.",Released
4,Jun 29 2022 10:35AM,240719,10,MSP211737,"Methapharm, Inc.",Released
5,Jun 29 2022 10:35AM,240720,15,AL-000001088,"Alexso, Inc.",Released
6,Jun 29 2022 10:34AM,240718,15,8205410,"Mizner Bioscience, LLC",Released
7,Jun 29 2022 10:25AM,240716,15,8205415,"Carwin Pharmaceutical Associates, LLC",Released
8,Jun 29 2022 10:25AM,240716,15,8208020,"Carwin Pharmaceutical Associates, LLC",Released
9,Jun 29 2022 10:25AM,240716,15,8208021,"Carwin Pharmaceutical Associates, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,240716,Released,4
40,240718,Released,1
41,240719,Released,4
42,240720,Released,1
43,240721,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240716,NaN,NaN,4.0
240718,NaN,NaN,1.0
240719,NaN,NaN,4.0
240720,NaN,NaN,1.0
240721,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240613,0.0,0.0,1.0
240623,0.0,0.0,1.0
240634,0.0,0.0,1.0
240635,0.0,0.0,1.0
240643,1.0,12.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240613,0,0,1
240623,0,0,1
240634,0,0,1
240635,0,0,1
240643,1,12,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240613,0,0,1
1,240623,0,0,1
2,240634,0,0,1
3,240635,0,0,1
4,240643,1,12,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240613,,,1
1,240623,,,1
2,240634,,,1
3,240635,,,1
4,240643,1,12,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 29 2022 10:37AM,240721,19,VITABIOTICS USA INC.
1,Jun 29 2022 10:35AM,240719,10,"Methapharm, Inc."
5,Jun 29 2022 10:35AM,240720,15,"Alexso, Inc."
6,Jun 29 2022 10:34AM,240718,15,"Mizner Bioscience, LLC"
7,Jun 29 2022 10:25AM,240716,15,"Carwin Pharmaceutical Associates, LLC"
11,Jun 29 2022 10:10AM,240715,10,Emerginnova
15,Jun 29 2022 10:08AM,240714,10,Beach Products Inc
36,Jun 29 2022 10:00AM,240713,10,"Citieffe, Inc."
37,Jun 29 2022 9:48AM,240711,19,"AdvaGen Pharma, Ltd"
40,Jun 29 2022 9:48AM,240653,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 29 2022 10:37AM,240721,19,VITABIOTICS USA INC.,,,1
1,Jun 29 2022 10:35AM,240719,10,"Methapharm, Inc.",,,4
2,Jun 29 2022 10:35AM,240720,15,"Alexso, Inc.",,,1
3,Jun 29 2022 10:34AM,240718,15,"Mizner Bioscience, LLC",,,1
4,Jun 29 2022 10:25AM,240716,15,"Carwin Pharmaceutical Associates, LLC",,,4
5,Jun 29 2022 10:10AM,240715,10,Emerginnova,,,4
6,Jun 29 2022 10:08AM,240714,10,Beach Products Inc,,,21
7,Jun 29 2022 10:00AM,240713,10,"Citieffe, Inc.",,,1
8,Jun 29 2022 9:48AM,240711,19,"AdvaGen Pharma, Ltd",,1,2
9,Jun 29 2022 9:48AM,240653,15,"Person & Covey, Inc.",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 10:37AM,240721,19,VITABIOTICS USA INC.,1,,
1,Jun 29 2022 10:35AM,240719,10,"Methapharm, Inc.",4,,
2,Jun 29 2022 10:35AM,240720,15,"Alexso, Inc.",1,,
3,Jun 29 2022 10:34AM,240718,15,"Mizner Bioscience, LLC",1,,
4,Jun 29 2022 10:25AM,240716,15,"Carwin Pharmaceutical Associates, LLC",4,,
5,Jun 29 2022 10:10AM,240715,10,Emerginnova,4,,
6,Jun 29 2022 10:08AM,240714,10,Beach Products Inc,21,,
7,Jun 29 2022 10:00AM,240713,10,"Citieffe, Inc.",1,,
8,Jun 29 2022 9:48AM,240711,19,"AdvaGen Pharma, Ltd",2,1,
9,Jun 29 2022 9:48AM,240653,15,"Person & Covey, Inc.",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 10:37AM,240721,19,VITABIOTICS USA INC.,1,,
1,Jun 29 2022 10:35AM,240719,10,"Methapharm, Inc.",4,,
2,Jun 29 2022 10:35AM,240720,15,"Alexso, Inc.",1,,
3,Jun 29 2022 10:34AM,240718,15,"Mizner Bioscience, LLC",1,,
4,Jun 29 2022 10:25AM,240716,15,"Carwin Pharmaceutical Associates, LLC",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 10:37AM,240721,19,VITABIOTICS USA INC.,1.0,NaN,NaN
1,Jun 29 2022 10:35AM,240719,10,"Methapharm, Inc.",4.0,NaN,NaN
2,Jun 29 2022 10:35AM,240720,15,"Alexso, Inc.",1.0,NaN,NaN
3,Jun 29 2022 10:34AM,240718,15,"Mizner Bioscience, LLC",1.0,NaN,NaN
4,Jun 29 2022 10:25AM,240716,15,"Carwin Pharmaceutical Associates, LLC",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 29 2022 10:37AM,240721,19,VITABIOTICS USA INC.,1.0,0.0,0.0
1,Jun 29 2022 10:35AM,240719,10,"Methapharm, Inc.",4.0,0.0,0.0
2,Jun 29 2022 10:35AM,240720,15,"Alexso, Inc.",1.0,0.0,0.0
3,Jun 29 2022 10:34AM,240718,15,"Mizner Bioscience, LLC",1.0,0.0,0.0
4,Jun 29 2022 10:25AM,240716,15,"Carwin Pharmaceutical Associates, LLC",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2888392,112.0,3.0,2.0
12,722091,1.0,4.0,0.0
15,1684748,62.0,0.0,0.0
16,240687,0.0,1.0,0.0
18,481320,2.0,0.0,0.0
19,1203412,33.0,5.0,0.0
20,721927,17.0,12.0,1.0
21,721948,3.0,0.0,0.0
22,240665,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2888392,112.0,3.0,2.0
1,12,722091,1.0,4.0,0.0
2,15,1684748,62.0,0.0,0.0
3,16,240687,0.0,1.0,0.0
4,18,481320,2.0,0.0,0.0
5,19,1203412,33.0,5.0,0.0
6,20,721927,17.0,12.0,1.0
7,21,721948,3.0,0.0,0.0
8,22,240665,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,112.0,3.0,2.0
1,12,1.0,4.0,0.0
2,15,62.0,0.0,0.0
3,16,0.0,1.0,0.0
4,18,2.0,0.0,0.0
5,19,33.0,5.0,0.0
6,20,17.0,12.0,1.0
7,21,3.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,112.0
1,12,Released,1.0
2,15,Released,62.0
3,16,Released,0.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,3.0,4.0,0.0,1.0,0.0,5.0,12.0,0.0,0.0
Released,112.0,1.0,62.0,0.0,2.0,33.0,17.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,3.0,4.0,0.0,1.0,0.0,5.0,12.0,0.0,0.0
2,Released,112.0,1.0,62.0,0.0,2.0,33.0,17.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,3.0,4.0,0.0,1.0,0.0,5.0,12.0,0.0,0.0
2,Released,112.0,1.0,62.0,0.0,2.0,33.0,17.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()